# Intro, Imaging & Machine Learning in PyCrystEM

PyCrystEM is an open-source library for crystallographic electron microscopy. Multi-dimensional data processing tools build on the HyperSpy library.

This notebook demonstrates the use of PyCrystEM to: form so called 'virtual diffraction images' by plotting the diffracted intensity in a particular pixel in the reciprocal space images, learn the component patterns making up the data using machine learning techniques, and perform data enhancement as a pre-processing step for further analysis such as indexation and orientation mapping. 

## Authors

08/06/17 Duncan Johnstone - Developed for Trondheim Diffraction Workshop

# Requirements

Pycrystem 0.1

HyperSpy 1.3

PyMatGen

## Contents

1. <a href='#loa'> Loading & Inspection</a>
2. <a href='#vdf'> Virtual Diffraction Imaging</a>
3. <a href='#ml'> Machine Learning SPED Data</a>
4. <a href='#pre'> Pre-processing & Peak Finding</a>

Import pycrystem

In [1]:
%matplotlib tk
import pycrystem as pc

C:\Users\Duncan\AppData\Local\conda\conda\envs\pycrystem\lib\site-packages\pymatgen\util\plotting_utils.py:3: UserWarning: pymatgen.util.plotting_utils has been renamed pymatgen.util.plotting. This stub will be removed in PMG 5.0.
  warnings.warn("pymatgen.util.plotting_utils has been renamed "


## <a id='loa'></a> 1. Loading and Inspection

Load the SPED data acquired from the nanowire

In [2]:
dp = pc.load('nanowire_precession.hdf5')

Look at what kind of object 'dp' is

In [3]:
dp

<ElectronDiffraction, title: , dimensions: (30, 100|144, 144)>

Inspect the metadata associated with the object 'dp'

In [4]:
dp.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── beam_energy = 300.0
│       ├── camera_length = 0.21000000000000002
│       └── scan_rotation = 277.0
├── General
│   ├── original_filename = nanowire_precession.blo
│   ├── time = (2014, 12, 8)
│   └── title = 
└── Signal
    ├── binned = False
    ├── signal_origin = 
    └── signal_type = electron_diffraction

Set important experimental parameters using the built in function

In [5]:
dp.set_experimental_parameters(accelerating_voltage=300.,
                               camera_length=40.,
                               scan_rotation=270.,
                               convergence_angle=2.,
                               exposure_time=10.)

See how this changed the metadata

In [6]:
dp.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── Diffraction
│       │       ├── camera_length = 40.0
│       │       └── exposure_time = 10.0
│       ├── accelerating_voltage = 300.0
│       ├── beam_energy = 300.0
│       ├── camera_length = 0.21000000000000002
│       ├── convergence_angle = 2.0
│       └── scan_rotation = 270.0
├── General
│   ├── original_filename = nanowire_precession.blo
│   ├── time = (2014, 12, 8)
│   └── title = 
└── Signal
    ├── binned = False
    ├── signal_origin = 
    └── signal_type = electron_diffraction

Set another metadata item and check it

In [ ]:
dp.metadata.set_item("General.title", 'GaAs Nanowire')
dp.metadata

Plot the data to inspect it

In [ ]:
dp.plot()

Set the diffraction pattern calibration

In [ ]:
dp.set_calibration(0.0035)

Plot the data with the new calibration

In [ ]:
dp.plot()

## <a id='vdf'></a> 2. Virtual Diffraction Imaging

Plot an interactive virtual image integrating intensity within a circular subset of pixels in the diffraction pattern

In [ ]:
roi = pc.roi.CircleROI(cx=0.,cy=0, r_inner=0, r=0.02)
dp.plot_interactive_virtual_image(roi=roi)

Get the virtual diffraction image associated with the last integration window used interactively

In [ ]:
vdf=dp.get_virtual_image(roi)

Plot the virtual dark-field image

In [ ]:
vdf.plot()

Save the virtual dark-field image as a 32bit tif

In [ ]:
vdf.change_dtype('float32')
vdf.save('vdfeg.tif')

Plot the data with an adjustable marker indicating where to crop the scan region

In [ ]:
reg = pc.roi.RectangularROI(left=50, top=837.5, right=290, bottom=1237.5)
dp.plot()
reg.add_widget(dp)

Crop the dataset based on the region defined above

In [ ]:
dpc = reg(dp)

# <a id='ml'></a> 3. Unsupervised learning of component patterns

Perform singular value decomposition (SVD) of the data

In [ ]:
dpc.change_dtype('float')
dpc.decomposition(True, algorithm='svd')

Obtain a "Scree plot" by plotting the fraction of variance described by each principal component

In [ ]:
dpc.plot_explained_variance_ratio()

Plot the decomposition results and have a look at them

In [ ]:
dpc.plot_decomposition_results()

Perform non-negative matrix factorisation (NMF)

In [ ]:
dpc.decomposition(True, algorithm='nmf', output_dimension=3)

Plot the NMF results

In [ ]:
dpc.plot_decomposition_results()

# <a id='pre'></a> 4. Pre-processing & Peak Finding

In [ ]:
dp = pc.ElectronDiffraction(dp)

Perform a background subtraction by radially integrating the 2D diffraction pattern and fitting a model containing a lorentzian, exponential and linear function

In [ ]:
dp.remove_background(1).plot()

Obtain a radially integrated diffraction profile and plot it

In [ ]:
rp = dp.get_radial_profile()
rp.plot()

Interactively tune peak finding parameters

In [ ]:
dp.find_peaks_interactive()

Perform peak finding on all diffraction patterns in data

In [ ]:
peaks = dp.find_peaks()

Check the peaks object

In [ ]:
peaks

Look at what's in the peaks object

In [ ]:
peaks.data